In [63]:
MODEL = "gpt-4o-2024-08-06"
SYS_MSG = """
You are a helpful research assistant.
    Users will ask a question regarding whether or not the paper presents evidence forward or against a given argument,
    You will output ONLY 2 things:
    - argument: the argument presented by the paper regarding the user's query, like: "Yes this paper validates this idea by discussing...." etc..
    - evidence: a list of DIRECT QUOTES from the paper containing information that support the argument above.
    """

In [13]:
from langchain_community.document_loaders import PyPDFLoader

def load_pdf_to_str(file_path: str) -> str:
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    return " ".join([page.page_content for page in docs])

file_path = "./assets-resources/agentcoder-paper.pdf"
pdf_contents = load_pdf_to_str(file_path)
pdf_contents

'AgentCoder: Multi-Agent Code Generation with\nEffective Testing and Self-optimisation\nDong Huang\nUniversity of Hong Kong\ndhuang@cs.hku.hkJie M.Zhang\njie.zhang@kcl.ac.uk\njie.zhang@kcl.ac.ukMichael Luck\nUniversity of Sussex\nMichael.Luck@sussex.ac.uk\nQingwen BU\nShanghai Jiao Tong University\nqwbu01@sjtu.edu.cnYuhao Qing\nUniversity of Hong Kong\nyhqing@cs.hku.hkHeming Cui\nUniversity of Hong Kong\nheming@cs.hku.hk\nAbstract\nAdvances in natural language processing (NLP) have been significantly boosted\nby the development of transformer-based large language models (LLMs). These\nmodels have revolutionized NLP tasks, particularly in code generation, aiding de-\nvelopers in creating software with enhanced efficiency. Despite their advances,\nchallenges remain in balancing code snippet generation with effective test cases.\nTo address these issues, this paper introduces AgentCoder, a novel code genera-\ntion solution comprising a multi-agent framework with a specialized test designe

In [65]:
from pydantic import BaseModel, Field
from typing import List


class Evidence(BaseModel):
    argument: str = Field(description="Argument forward, against or neutral regarding the question from the user.")
    evidence: List[str] = Field(description="List of DIRECT QUOTES serve as evidence to support the argument.") 

In [66]:
from openai import OpenAI

client = OpenAI()

def get_response(prompt_question):
    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "system", "content": SYS_MSG},
                  {"role": "user", "content": prompt_question}],
        response_format=Evidence,
    )
    
    return response.choices[0].message.parsed

In [67]:
prompt = """
Does this paper: 
{pdf_contents}\n\n
presents evidence for the following statement:
"The use of a multi-agent system, with roles for code generation, test design, and test execution can lead to improved efficiency and effectiveness in code generation tasks compared to single-agent models."
"""

output = get_response(prompt)

output

Evidence(argument='Yes, this paper supports the statement by discussing how a multi-agent system with distinct roles for code generation, test design, and test execution improves both the efficiency and effectiveness of code generation tasks.', evidence=['"Our experiments demonstrate that dividing tasks among specialized agents leads to more efficient workflows, as the code generation agent can focus solely on producing code without being burdened by testing tasks."', '"By allocating test design and execution to dedicated agents, the system capitalizes on their respective strengths, ensuring that code is not only generated quickly but also meets higher quality standards."', '"In comparison to single-agent models, the multi-agent approach significantly reduced code generation time while simultaneously increasing the quality of the produced code as verified by test agents."', '"The collaboration between agents in our multi-agent system fosters an environment where code generation is not 

In [68]:
output.argument

'Yes, this paper supports the statement by discussing how a multi-agent system with distinct roles for code generation, test design, and test execution improves both the efficiency and effectiveness of code generation tasks.'

In [69]:
output.evidence

['"Our experiments demonstrate that dividing tasks among specialized agents leads to more efficient workflows, as the code generation agent can focus solely on producing code without being burdened by testing tasks."',
 '"By allocating test design and execution to dedicated agents, the system capitalizes on their respective strengths, ensuring that code is not only generated quickly but also meets higher quality standards."',
 '"In comparison to single-agent models, the multi-agent approach significantly reduced code generation time while simultaneously increasing the quality of the produced code as verified by test agents."',
 '"The collaboration between agents in our multi-agent system fosters an environment where code generation is not only faster but also more robust, as evidenced by higher success rates in code validation tasks compared to single-agent baseline systems."]}]}The results indicate that efficiency in code generation can be substantially improved."']

In [50]:
from IPython.display import Markdown


Markdown(output.argument)

Yes this paper validates this idea by discussing the advantages of multi-agent systems over single-agent models.

In [54]:
for evidence in output.evidence:
    print(evidence)
    # if evidence in pdf_contents:
    #     print(evidence)
    # else:
    #     print("The quote was not found in the paper")

"The implementation of a multi-agent system allows for distinct roles such as code generation, test design, and test execution, enabling parallel processing and specialization, which significantly enhances overall efficiency."
"Research indicates that multi-agent systems outperform single-agent models in collaborative environments, particularly in tasks requiring simultaneous processing of multiple aspects of software development."
"By distributing responsibilities among multiple agents, the system improves the effectiveness of testing and code generation, as agents can focus on their specific tasks without the bottleneck associated with single-agent approaches."


NOT WORKING YET